In [1]:
import sys
sys.path.append("/tmp/pycharm_project_754")
base = "/tmp/pycharm_project_754"
data_base = base + '/data/'
out_base = base + '/out/'

In [ ]:
import scipy.io
import urllib.request
import os
from shutil import copyfile

data_url = 'https://s3.us-east-2.amazonaws.com/dgl.ai/dataset/ACM.mat'
data_file_path = '/tmp/ACM.mat'
if not os.path.exists(data_file_path) : 
    print("data file is not existed, download now")
    urllib.request.urlretrieve(data_url, data_file_path)
    copyfile(data_file_path, data_base + 'test/ACM.mat')  
print("data file prepare done")
data = scipy.io.loadmat(data_base + 'test/ACM.mat')
print(list(data.keys()))

data file is not existed, download now


In [4]:
print(type(data['PvsA']))
print('#Papers:', data['PvsA'].shape[0])
print('#Authors:', data['PvsA'].shape[1])
print('#Links:', data['PvsA'].nnz)

<class 'scipy.sparse.csc.csc_matrix'>
#Papers: 12499
#Authors: 17431
#Links: 37055


In [5]:
# build heterograph
import dgl
pa_g = dgl.heterograph({('paper', 'written-by', 'author') : data['PvsA']})


In [6]:
print('Node types:', pa_g.ntypes)
print('Edge types:', pa_g.etypes)
print('Canonical edge types:', pa_g.canonical_etypes)

# Nodes and edges are assigned integer IDs starting from zero and each type has its own counting.
# To distinguish the nodes and edges of different types, specify the type name as the argument.
print(pa_g.number_of_nodes('paper'))
# Canonical edge type name can be shortened to only one edge type name if it is
# uniquely distinguishable.
print(pa_g.number_of_edges(('paper', 'written-by', 'author')))
print(pa_g.number_of_edges('written-by'))
print(pa_g.successors(1, etype='written-by'))  # get the authors that write paper #1

# Type name argument could be omitted whenever the behavior is unambiguous.
print(pa_g.number_of_edges())  # Only one edge type, the edge type argument could be omitted

Node types: ['paper', 'author']
Edge types: ['written-by']
Canonical edge types: [('paper', 'written-by', 'author')]
12499
37055
37055
tensor([3532, 6421, 8516, 8560])
37055


In [ ]:
print(1)

In [ ]:
G = dgl.heterograph({
        ('paper', 'written-by', 'author') : data['PvsA'],
        ('author', 'writing', 'paper') : data['PvsA'].transpose(),
        ('paper', 'citing', 'paper') : data['PvsP'],
        ('paper', 'cited', 'paper') : data['PvsP'].transpose(),
        ('paper', 'is-about', 'subject') : data['PvsL'],
        ('subject', 'has', 'paper') : data['PvsL'].transpose(),
    })

print(G)

In [ ]:
G.nodes['paper'].shape
# not supported in windows

# Draw the metagraph using graphviz.
# import pygraphviz as pgv
# def plot_graph(nxg):
#     ag = pgv.AGraph(strict=False, directed=True)
#     for u, v, k in nxg.edges(keys=True):
#         ag.add_edge(u, v, label=k)
#     ag.layout('dot')
#     ag.draw(out_base + '/graph.png')
# 
# plot_graph(G.metagraph)

In [28]:
# dia matrix: 
# 做矩阵乘积较快的方式, 适合求特征值, svd分解什么的. 但是如果矩阵的对角性不好, 
# 矩阵大小就要比其他存储方式高到不知道哪里去了. (所以random矩阵最好不要用这个方式)
# coo/csc/csr matrix: 
# 存储方式相对的比较灵活, 也好理解. 操作重点在于读写矩阵的话用这个可能会比较好.
# bsr matrix 分块存储
# dok matrix: 基于键值对的方式存储的矩阵, 有点像字典的存储方式
# lil matrix: 很有意思的增量式存储方式, 文档里面说这种方式对索引, 
# 切片操作的资瓷是非常吼的.

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
# pvc shape (12499, 14) 14 conference and 12499 paper
pvc = data['PvsC'].tocsr()
# find all papers published in KDD, ICML, VLDB
c_selected = [0, 11, 13]  # KDD, ICML, VLDB
p_selected = pvc[:, c_selected].tocoo()
# generate labels
labels = pvc.indices # 0 KDD
labels[labels == 11] = 1 # 1 ICML
labels[labels == 13] = 2 # 2 VLDB

labels = torch.tensor(labels).long().to(torch.device('cuda:0'))

# generate train/val/test split
pid = p_selected.row
shuffle = np.random.permutation(pid)
train_idx = torch.tensor(shuffle[0:800]).long().to(torch.device('cuda:0'))
val_idx = torch.tensor(shuffle[800:900]).long().to(torch.device('cuda:0'))
test_idx = torch.tensor(shuffle[900:]).long().to(torch.device('cuda:0'))


In [30]:
print(labels)

tensor([0, 0, 0,  ..., 7, 7, 7], device='cuda:0')


In [10]:
import GCN_model as gmodel
model = gmodel.HeteroRGCN(G, 10, 10, 3)

In [41]:
model.to(torch.device('cuda:0'))


HeteroRGCN(
  (embed): ParameterDict(
      (author): Parameter containing: [torch.cuda.FloatTensor of size 17431x10 (GPU 0)]
      (paper): Parameter containing: [torch.cuda.FloatTensor of size 12499x10 (GPU 0)]
      (subject): Parameter containing: [torch.cuda.FloatTensor of size 73x10 (GPU 0)]
  )
  (layer1): HeteroRGCNLayer(
    (weight): ModuleDict(
      (cited): Linear(in_features=10, out_features=10, bias=True)
      (citing): Linear(in_features=10, out_features=10, bias=True)
      (has): Linear(in_features=10, out_features=10, bias=True)
      (is-about): Linear(in_features=10, out_features=10, bias=True)
      (writing): Linear(in_features=10, out_features=10, bias=True)
      (written-by): Linear(in_features=10, out_features=10, bias=True)
    )
  )
  (layer2): HeteroRGCNLayer(
    (weight): ModuleDict(
      (cited): Linear(in_features=10, out_features=3, bias=True)
      (citing): Linear(in_features=10, out_features=3, bias=True)
      (has): Linear(in_features=10, out_f

In [42]:
opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
best_val_acc = 0
best_test_acc = 0

for epoch in range(100):
    logits = model(G)
    # The loss is computed only for labeled nodes.
    loss = criterion(logits[train_idx], labels[train_idx])

    pred = logits.argmax(1)
    train_acc = (pred[train_idx] == labels[train_idx]).float().mean()
    val_acc = (pred[val_idx] == labels[val_idx]).float().mean()
    test_acc = (pred[test_idx] == labels[test_idx]).float().mean()

    if best_val_acc < val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc

    opt.zero_grad()
    loss.backward()
    opt.step()

    if epoch % 5 == 0:
        print('Loss %.4f, Train Acc %.4f, Val Acc %.4f (Best %.4f), Test Acc %.4f (Best %.4f)' % (
            loss.item(),
            train_acc.item(),
            val_acc.item(),
            best_val_acc.item(),
            test_acc.item(),
            best_test_acc.item(),
        ))



Loss 1.3115, Train Acc 0.1512, Val Acc 0.1500 (Best 0.1500), Test Acc 0.1382 (Best 0.1382)
Loss 0.9693, Train Acc 0.5650, Val Acc 0.4600 (Best 0.4600), Test Acc 0.5226 (Best 0.5226)
Loss 0.8332, Train Acc 0.5250, Val Acc 0.4300 (Best 0.4600), Test Acc 0.5075 (Best 0.5226)
Loss 0.6297, Train Acc 0.8237, Val Acc 0.6400 (Best 0.6400), Test Acc 0.6625 (Best 0.6625)
Loss 0.4157, Train Acc 0.9250, Val Acc 0.7400 (Best 0.7400), Test Acc 0.7194 (Best 0.7194)
Loss 0.2462, Train Acc 0.9600, Val Acc 0.7600 (Best 0.7600), Test Acc 0.7328 (Best 0.7286)
Loss 0.1471, Train Acc 0.9700, Val Acc 0.7500 (Best 0.7600), Test Acc 0.7395 (Best 0.7286)
Loss 0.1027, Train Acc 0.9750, Val Acc 0.7600 (Best 0.7600), Test Acc 0.7404 (Best 0.7286)
Loss 0.0828, Train Acc 0.9762, Val Acc 0.8000 (Best 0.8000), Test Acc 0.7462 (Best 0.7462)
Loss 0.0724, Train Acc 0.9762, Val Acc 0.7900 (Best 0.8100), Test Acc 0.7462 (Best 0.7471)
Loss 0.0661, Train Acc 0.9762, Val Acc 0.7800 (Best 0.8100), Test Acc 0.7462 (Best 0.7471)

In [35]:
pred = logits.argmax(1)

In [39]:
print(logits[0])
print(labels[0])

tensor([ 2.7476, -3.2470, -1.2180], device='cuda:0', grad_fn=<SelectBackward>)
tensor(0, device='cuda:0')


In [44]:
print(G.canonical_etypes)

[('paper', 'written-by', 'author'), ('author', 'writing', 'paper'), ('paper', 'citing', 'paper'), ('paper', 'cited', 'paper'), ('paper', 'is-about', 'subject'), ('subject', 'has', 'paper')]


In [45]:
'Wh_%s' % 'is-about'



'Wh_is-about'

In [7]:
G.nodes['paper'].data['h'][0]


KeyError: 'h'

In [21]:
print(sum([G.number_of_nodes(name) for name in G.ntypes]))


30003


In [55]:
import importlib
importlib.reload(gmodel)

<module 'GCN_model' from '/tmp/pycharm_project_754/GCN_model.py'>

In [56]:
G.to(torch.device('cpu'))
nmodel = gmodel.NormalGCN(G, 10, 10, 3)
nmodel.to(torch.device('cpu'))


cpu
cpu
cpu


NormalGCN(
  (layer1): NormalGCNLayer(
    (weight): Linear(in_features=10, out_features=10, bias=True)
  )
  (layer2): NormalGCNLayer(
    (weight): Linear(in_features=10, out_features=3, bias=True)
  )
)

In [58]:
labels = torch.tensor(labels).long()

# generate train/val/test split
pid = p_selected.row
shuffle = np.random.permutation(pid)
train_idx = torch.tensor(shuffle[0:800]).long()
val_idx = torch.tensor(shuffle[800:900]).long()
test_idx = torch.tensor(shuffle[900:]).long()
opt = torch.optim.Adam(nmodel.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
best_val_acc = 0
best_test_acc = 0

for epoch in range(100):
    logits = nmodel(G)
    # The loss is computed only for labeled nodes.
    loss = criterion(logits[train_idx], labels[train_idx])

    pred = logits.argmax(1)
    train_acc = (pred[train_idx] == labels[train_idx]).float().mean()
    val_acc = (pred[val_idx] == labels[val_idx]).float().mean()
    test_acc = (pred[test_idx] == labels[test_idx]).float().mean()

    if best_val_acc < val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc

    opt.zero_grad()
    loss.backward()
    opt.step()

    if epoch % 5 == 0:
        print('Loss %.4f, Train Acc %.4f, Val Acc %.4f (Best %.4f), Test Acc %.4f (Best %.4f)' % (
            loss.item(),
            train_acc.item(),
            val_acc.item(),
            best_val_acc.item(),
            test_acc.item(),
            best_test_acc.item(),
        ))





/data/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


KeyError: tensor([ 5620,    61,  8968,  6083,  5669,  5683,  2716,  1389,  7579,  6053,
         6143,  2692,  8702,  5687,  7294, 10725,  5640,     3,  5747,  8727,
         8461,  6044,  1546, 10743, 10241,  8737,  6821,  5688,  7311,  7232,
         2770,  6533,    44,  1381,  1557,  2206,  6580,  8437,  5905,  5646,
         2287,  8538,  8675,  8424,    21,  1600,  2310,  2719,  9888,  5718,
         8969,  1555,  6003,  7594,  1297,  6171, 10703,  2298,  9876,  7580,
        10776,  7260,  8681,  6182,  6199,  8482,    49,  7610, 10216,  7300,
         2700,  1613,  6161,  8467,  9879,  5691,  8427,  8985,  6597,  5936,
         1360,  9847,  7269, 10251,  1356,  6178,  6814,  2221,  5629,  2701,
          116,  5668,  5974,  8484,  1619,  2682,  8726,  5947,  1584,  1363,
         5625,  5980,  2731,  5679,  7297,  7567,  5910,  9850, 10720,  8963,
         8442,  5730,  5741,  8490,  2708,  7238,  6775, 10706,  6802,  7256,
         1596,  7595,  1639,  6767,  2699,  8942,  654